<p style="text-align: center;"><font size="8"><b>Defining Our Own Classes</b></font><br>


Python is an object oriented programming language. Remember from earlier how we defined objects and classes. An object in a single instance of a class. We have already several examples of classes. Remember backs to lists.

In [3]:
groceries = list()
type(groceries)

list

`groceries` is a *object* of type *list*. List is one of the classes built in to Python. A list has *attributes*, for example a length. It also has *methods*, for example `append`, or `pop`.

Today we will look at designing our own classes. Classes can be used for many tasks. All of the algorithms we have already seen, for example Newton's method, or LCG random number generators could be written as part of a class, although we wrote them as standalone functions.

This illustrates an important point. Python supports classes, but unlike other object oriented languages (for example Java) does not require them. You are likely proficient enough by now in Python to code up an algorithm without classes. So why do we need classes at all?

Classes are a nice way to group together a set of data and functions that operate on that data. This leads to modular code with more manageable (i.e. smaller) units. Even though you do not need functions, they often provide a more elegant solution that is easier to extend late on.

In addition, outside the world of mathematical programming, classes can be a very natural way of thinking about problems. Most modern software development is based on classes and objects.

# A Function Class

One of the most frequent uses of classes in scientific computing is to represent mathematical functions that have a set of parameters in addition to one or more independent variables. For example, consider the trajectory $y(t)$ of a ball thrown straight up in the air,
$$ y(t) = v_0 t - \frac{1}{2}gt^2.$$
Here $v_0$ is the initial velocity of the ball (from the throw) and $g$ is the acceleration due to gravity. 

Notice that we have written $y(t)$, i.e. $y$ as a function of time, even though clearly $y$ also depends upon $v_0$ and $g$ as well.  We could equally write $y(t; v_0, g)$.

To code this up we could write:

In [6]:
def y(t,v0,g=9.81):
    return v0*t - 0.5*g*t**2

Why is this a problem? 

Remember that earlier we wrote a function `left_reimann_sum` that takes in a function as an argument:

In [7]:
def left_reimann_sum(f, a, b, N):
    import numpy as np
    x = np.linspace(a,b,N+1)
    h = (b - a)/N
    
    return h*np.sum(f(x))

The input `f` is a function of a single variable. If we pass in our function `y` we get an error.

In [8]:
left_reimann_sum(y, 0, 1, 100)

TypeError: y() missing 1 required positional argument: 'v0'

The function `y` we defined earlier requires at least one additional argument, $v_0$. This means that `y` as written cannot be passed into our `left_reimann_sum` function

One possible solution is to rewrite `left_reimann_sum` to allow for functions to take two arguments. If we do this however, `left_reimann_sum` would no longer work with functions of a single variable; and it still wouldn't work with functions that take in more than two variables.

A better solution is to write $y(t)$ as a class.

As mentioned earlier, a class is a set of variables and a set of functions held together as one unit. The variables are visible in all functions in the class. This is also true of modules, however classes are very different from modules. For example you can make make copies of a class, while there can only be one copy of a module. As you become more familiar with classes you will see the differences between them and modules and where it makes sense to use one over the other.

Consider as before the function 
$$ y(t) = v_0 t - \frac{1}{2}gt^2.$$

We can say that $v_0$ and $g$ are the data. A function, for example `value(t)` is needed to evaluate $y$ at time $t$. Clearly `value` must also have access to $v_0$ and $g$. 

Without further ado, here's the class:

In [50]:
class Y:
    def __init__(self, v0, g=9.81):
        self.v0 = v0
        self.g = g
        
    def value(self, t):
        return self.v0*t - 0.5*self.g*t**2

Class always start with the syntax `class name:`, where `name` is the name of the class. 

Within the body of the class we must define all the methods (a.k.a. member functions) that are supported by the class.

In addition to the function value, you'll see that we defined another function called `__init__`. This is known as the constructor. This function initializes the data. Most functions contain a constructor, and it always goes by the name `__init__`. Each time the caller initiates the class, the constructor is called automatically by Python.

You'll notice that the first parameter to all the memeber functions is the mysterious variable `self`. This is an implicit parameter that serves internally to identify and store the particular instance being constructed. There may be several different objects of type `Y`, each having its state stored in memory. 

The `self` identifier allow us to access members of this instance using the standard form `object.membername`. For example we set `self.v0 = v0` to set the value of `v0`. Later on in the `value` function we access the value of `v0` by calling `self.v0`.

If, in the initializer we had called `v0=v0` for example, we would be establishing `v0` as a local variable within the function body and we would be unable to access it later.

A little more on the `self` variable:
* any class method must have `self` as the first argument
* self represents an instance of the class
* to access another class method or attribute inside class methods, we must prefix with `self`
* `self` is dropped as an argument in calls to class methods

This can be confusing at first, but with a little hands on experience it will make sense.

Let's look at how to use this class.

In [51]:
y = Y(3) # create object with v0 = 3
type(y)  # display object type

__main__.Y

We can now input `y.value` as a function to `left_reimann_sum`.

In [52]:
left_reimann_sum(y.value, 0, 1, 100)

-0.14460675000000009

We can create more instances of this class

In [58]:
y1 = Y(2)
y2 = Y(-1)

print(left_reimann_sum(y1.value,0,1,100))
print(left_reimann_sum(y2.value,0,1,100))

-0.64960675
-2.16460675


## Doc Strings

As with functions, classes can and should have doc strings. For example a doc string for the `Y` class might look like:

In [59]:
class Y:
    """
    Class representing a function describing the vertical trajectory of a ball
    
    """
    def __init__(self, v0, g=9.81):
        """
        Constructor
        
        Parameters
        ----------
        v0 : float
            initial vertical velocity [m/s]
        g  : float, default 9.81
            acceleration due to gravity [m/s^2]
            
        """
        self.v0 = v0
        self.g = g
        
    def value(self, t):
        """
        Compute the velocity at time t
        
        Parameters
        ----------
        t : float
            time since intitial throw [s]
            
        Returns
        ------
        y : float
            vertical distance ball has travelled [m]
            
        """
        return self.v0*t - 0.5*self.g*t**2

Calling the help function now returns:

In [60]:
help(Y)

Help on class Y in module __main__:

class Y(builtins.object)
 |  Class representing a function describing the vertical trajectory of a ball
 |  
 |  Methods defined here:
 |  
 |  __init__(self, v0, g=9.81)
 |      Constructor
 |      
 |      Parameters
 |      ----------
 |      v0 : float
 |          initial vertical velocity [m/s]
 |      g  : float, default 9.81
 |          acceleration due to gravity [m/s^2]
 |  
 |  value(self, t)
 |      Compute the velocity at time t
 |      
 |      Parameters
 |      ----------
 |      t : float
 |          time since intitial throw [s]
 |          
 |      Returns
 |      ------
 |      y : float
 |          vertical distance ball has travelled [m]
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [61]:
print(y1.__dict__)

{'g': 9.81, 'v0': 2}


In [62]:
print(y1.__weakref__)

None


# Special Methods

Some class methods have names starting and ending with a double underscore. We've already seen the `__init__` function for example. This constructor is called automatically when an instance is created. 

Other special methods allow us to perform arithmetic operations with other instances, for example "+", "-", "\*" etc. 

Other languages may call this operator overloading.

## Call Method

Computing the value of the mathematical function represented by the class `Y` is done by calling the function `y.value(t)` (y being the name of an instance). If we could call `y(t)` instead this would look more like an ordinary function. 

Such a syntax is possible using the `__call__` special method. 

This implementation of `Y` would look like:

In [66]:
class Y:
    def __init__(self, v0, g=9.81):
        self.v0 = v0
        self.g = g
        
    def __call__(self, t):
        return self.v0*t - 0.5*self.g*t**2

We can now use `Y` directly in our `left_reimann_sum` function:

In [68]:
y = Y(2)
left_reimann_sum(y, 0, 1, 100)

-0.64960675000000012

Note that we no longer need a `value` function. A good convention is to include a `__call__` method in all classes that represent a mathematical function.

Objects that include a `__call__` method are said to be *callable objects*. Plain functions are also called callable. 

The function `callable(a)` tests  whether `a` behaves as a callable, i.e. if `a` is a function or an object with a `__call__` method.

In [69]:
callable(y)

True

## Adding Objects

Let `a` and `b` be instances of some class `C`. Can we write `a + b`? Yes, if class `C` has a special method, `__add__`. 

    class C:
    ....
        __add__(self, other):
        .....
        
The `__add__` method should add the instances `self` and `other` (whatever that means for the class `C`) and return the result as an instance. 

When Python encounters `a+b` it will check if class `C` contains an `__add__` method and if it does interpret `a + b` as `a.__add__(b)`.

## A Polynomial Class

Let's create a class that represents a polynomial. A polynomial is a function $p(x)$ of the form:
$$ p(x) = a_0 + a_1 x + a_2 x^2 + ... + a_n x^n.$$

We'll add a `__call__` function and an `__add__` function. 

Our polynomial class will take a list of coefficients $a_i$, $0\leq  i \leq n$. 

In [75]:
class Polynomial:
    def __init__(self, coefficients):
        self.coeff = coefficients
        
    def __call__(self, x):
        s = 0
        for i in range(len(self.coeff)):
            s += self.coeff[i]*x**i
            
        return s
    
    def __add__(self, other):
        # start with the longer list of coefficents and add the other list
        if len(self.coeff) > len(other.coeff):
            sum_coeff = self.coeff
            for i in range(len(other.coeff)):
                sum_coeff[i] += other.coeff[i]
                
        else:
            sum_coeff = other.coeff
            for i in range(len(self.coeff)):
                sum_coeff[i] += self.coeff[i]
                
        return Polynomial(sum_coeff)

In [76]:
p1 = Polynomial([1, 2, 5])
p2 = Polynomial([2, 2.4, 9, 0, -9.2])

p3 = p1 + p2
p3(5.2)

-6322.246720000001

## Turning an Instance Into a String

The `__str__` method converts an object to a string. We've seen this already for integers:

In [80]:
a = 1
b = str(a)
print("b has value",b,"and type",type(b))

b has value 1 and type <class 'str'>


Let's implement the `__str__` method for our Polynomial class. What do we want this method to do? 

For example, let's say we have a Polynomial object representing:
$$ p(x) = 2x^2 + 5x + 1.$$

Then we'd like the `__str__` method to return:

     2*x^2 + 5*x + 1

In [84]:
class Polynomial:
    def __init__(self, coefficients):
        self.coeff = coefficients
        
    def __call__(self, x):
        s = 0
        for i in range(len(self.coeff)):
            s += self.coeff[i]*x**i
            
        return s
    
    def __str__(self):
        s = ""
        for i in range(len(self.coeff)):
            s += " + %g*x^%d" % (self.coeff[i], i)
        return s
    
    def __add__(self, other):
        # start with the longer list of coefficents and add the other list
        if len(self.coeff) > len(other.coeff):
            sum_coeff = self.coeff
            for i in range(len(other.coeff)):
                sum_coeff[i] += other.coeff[i]
                
        else:
            sum_coeff = other.coeff
            for i in range(len(self.coeff)):
                sum_coeff[i] += self.coeff[i]
                
        return Polynomial(sum_coeff)

In [86]:
p = Polynomial([1,5,2])
print(str(p))

 + 1*x^0 + 5*x^1 + 2*x^2


## Other Special Methods

Given two objects `a` and `b` the standard arithmetic operators are defined by the following special methods:
* `a + b` : `a.__add__(b)`
* `a - b` : `a.__sub__(b)`
* `a * b` : `a.__mul__(b)`
* `a / b` : `a.__div__(b)`
* `a ** b` : `a.__pow__(b)`

In addition, there are other non arithmetic special methods:
* `len(a)` : `a.__len__()`
* `abs(a)` : `a.__abs__()`
* `a == b` : `a.__eq__(b)`
* `a > b` : `a.__gt__(b)`
* `a >= b` : `a.__ge__(b)`
* `a < b` : `a.__lt__(b)`
* `a <= b` : `a.__le__(b)`
* `a != b` : `a.__ne__(b)`
* `-a`     : `a.__neg__()`

There is also a boolean method `__bool__()` which returns true or false and allows us to call `if a` for example.